# Generate Test Results

## Prepare Test Images

In [1]:
from utils.prepare_images import prepare_images
import os
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
# put videos here!
video_base = 'data/videos'
#video_base = 'D:/e6691/6691_assignment2/videos'
videos = [vid for vid in os.listdir(video_base) if vid.endswith('mp4')]
# images will be output to here
image_base = 'data/images'
#image_base = 'D:/e6691/6691_assignment2/images'

In [28]:
test_videos = sorted(videos)[1:2]
test_videos

['RALIHR_surgeon01_fps01_0002.mp4']

In [29]:
# command line: 
# ffmpeg -i {input_video} -r {frame_rate} [-f {force_format} (not needed)] {output_images}
# doc: https://ffmpeg.org/ffmpeg.html
for video in test_videos:
    input_path = os.path.join(video_base, video)
    # make dirs
    output_base = image_base + '/{}'.format(video.split('.')[0])
    if not os.path.exists(output_base):
        os.mkdir(output_base)
    output_path = os.path.join(output_base, '%d.png')
    # # command
    print('Frames extracted from {} to {}'.format(input_path, output_path))
    # !!!!!! BECAREFUL WITH THIS LINE !!!!!!
    #!ffmpeg -i {input_path} -r 1 {output_path}

Frames extracted from data/videos/RALIHR_surgeon01_fps01_0002.mp4 to data/images/RALIHR_surgeon01_fps01_0002/%d.png


In [30]:
labels_path = 'data/labels/video.phase.trainingData.clean.StudentVersion.csv'
names_path = 'data/labels/all_labels_hernia.csv'
labels_df = pd.read_csv(labels_path)
names_df = pd.read_csv(names_path, index_col='id')

In [31]:
# all labels are set to 14
prepare_images(test_videos, video_base, image_base, labels_df, names_df)

/Users/harrypotter/Documents/courses/dl/assignment2/e6691-2022spring-assign2-vcsz/utils/prepare_images.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df[['StartSec', 'EndSec']] = video_df[['Start', 'End']].applymap(time2int)
/Users/harrypotter/Documents/courses/dl/assignment2/e6691-2022spring-assign2-vcsz/utils/prepare_images.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df[['StartSec', 'EndSec']] = video_df[['Start', 'End']].applymap(time2int)


## Predict

In [12]:
from models.SVRCNet.svrc import SVRC
from utils.trainer import Evaluator
from utils.read_videos import read

import torch
from torchvision import transforms

print(torch.version.cuda)

None


In [33]:
image_paths, labels = read(test_videos, image_base, ind_end=60)

In [34]:
X_test = sum(image_paths, [])
y_test = sum(labels, [])

In [35]:
list(zip(X_test, y_test))

[('data/images/RALIHR_surgeon01_fps01_0002/1-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/2-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/3-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/4-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/5-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/6-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/7-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/8-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/9-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/10-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/11-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/12-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/13-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/14-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/15-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/16-12.png', 12),
 ('data/images/RALIHR_surgeon01_fps01_0002/17-13.

In [36]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = SVRC()
if torch.cuda.is_available():
    model.to(device)

# this does not necessarily affect what mode we're running at
# weights of resnet is not changed anyway
# !!!!!! DELETE THIS LINE !!!!!!
WeightsPath_LSTM = 'models/weights_resnet18_70_LSTM'
model.load_state_dict(torch.load(WeightsPath_LSTM, map_location=device))

<All keys matched successfully>

In [37]:
data_transform = {
    "train": transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])
}

In [38]:
evaluator = Evaluator(model, device)

In [39]:
preds_res = evaluator.predict(X_test, data_transform['train'], pretrain=True)

In [40]:
preds_lstm = evaluator.predict(X_test, data_transform['train'], pretrain=False)

In [41]:
# seems that lstm does do something in terms of smoothing the predictions
list(zip(preds_res, preds_lstm))

[(13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (13, 13),
 (6, 6),
 (6, 6),
 (13, 13),
 (6, 13),
 (9, 13),
 (9, 13),
 (9, 11),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (9, 6),
 (9, 6),
 (9, 11),
 (6, 6),
 (9, 11),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (13, 6),
 (13, 13),
 (13, 13),
 (13, 13),
 (9, 13),
 (6, 6),
 (6, 13),
 (6, 6),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (6, 13),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (13, 6),
 (13, 6),
 (13, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6,

In [56]:
from scipy.signal import medfilt

In [57]:
preds_res_filtered = medfilt(preds_lstm, 3)
preds_res_filtered

array([13, 13, 13, ...,  5,  5,  5])

In [59]:
evaluator.eval(preds_res, y_test)
evaluator.eval(preds_lstm, y_test)
evaluator.eval(preds_res_filtered, y_test)

Accuracy: 0.15884928080050031
Accuracy: 0.16416510318949343
Accuracy: 0.16447779862414008


0.16447779862414008

## Convert to csv

In [ ]:
from parse import parse

In [50]:
template_path = 'data/labels/kaggle_template.csv'

In [54]:
template_df = pd.read_csv(template_path, names=['Predicted'])
template_df

,Predicted
Id,Predicted
001-0071-00001,NaN
001-0071-00002,NaN
001-0071-00003,NaN
001-0071-00004,NaN
...,...
003-0001-05902,NaN
003-0001-05903,NaN
003-0001-05904,NaN
003-0001-05905,NaN


In [62]:
# a module to parese image path
#!pip install parse

  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=6d291cbd38081a79135d91dd7611b263ba62ccfb53a69817a0ad675ff790d1a9
  Stored in directory: /Users/harrypotter/Library/Caches/pip/wheels/d6/9c/58/ee3ba36897e890f3ad81e9b730791a153fce20caa4a8a474df
Successfully built parse


In [73]:
parse('{}/RALIHR_surgeon{}_fps01_{}/{}-{}.png', X_test[0])[1:4]

('01', '0002', '1')

In [112]:
def format_path(s):
    ss = parse('{}/RALIHR_surgeon{}_fps01_{}/{}-{}.png', s)[1:4]
    return '{}-{}-{}'.format(
        ss[0].rjust(3, '0'), 
        ss[1].rjust(4, '0'), 
        ss[2].rjust(5, '0')
    )

In [113]:
names_df.loc[preds_res_filtered].values.flatten()

array(['transitionary idle', 'transitionary idle', 'transitionary idle',
       ..., 'out of body', 'out of body', 'out of body'], dtype=object)

In [114]:
test_df = pd.DataFrame({
    'preds': names_df.loc[preds_res_filtered].values.flatten()
}, index=list(map(format_path, X_test)))
test_df

,preds
001-0002-00001,transitionary idle
001-0002-00002,transitionary idle
001-0002-00003,transitionary idle
001-0002-00004,transitionary idle
001-0002-00005,transitionary idle
...,...
001-0002-03194,out of body
001-0002-03195,out of body
001-0002-03196,out of body
001-0002-03197,out of body


In [115]:
template_df.index

Index(['Id', '001-0071-00001', '001-0071-00002', '001-0071-00003',
       '001-0071-00004', '001-0071-00005', '001-0071-00006', '001-0071-00007',
       '001-0071-00008', '001-0071-00009',
       ...
       '003-0001-05897', '003-0001-05898', '003-0001-05899', '003-0001-05900',
       '003-0001-05901', '003-0001-05902', '003-0001-05903', '003-0001-05904',
       '003-0001-05905', '003-0001-05906'],
      dtype='object', length=179869)

In [116]:
test_df.index

Index(['001-0002-00001', '001-0002-00002', '001-0002-00003', '001-0002-00004',
       '001-0002-00005', '001-0002-00006', '001-0002-00007', '001-0002-00008',
       '001-0002-00009', '001-0002-00010',
       ...
       '001-0002-03189', '001-0002-03190', '001-0002-03191', '001-0002-03192',
       '001-0002-03193', '001-0002-03194', '001-0002-03195', '001-0002-03196',
       '001-0002-03197', '001-0002-03198'],
      dtype='object', length=3198)

In [117]:
# select test_df with template
preds_df = test_df.loc[test_df.index] #[template_df.index]
preds_df

,preds
001-0002-00001,transitionary idle
001-0002-00002,transitionary idle
001-0002-00003,transitionary idle
001-0002-00004,transitionary idle
001-0002-00005,transitionary idle
...,...
001-0002-03194,out of body
001-0002-03195,out of body
001-0002-03196,out of body
001-0002-03197,out of body


In [ ]:
preds_df.to_csv('data/labels/kaggle_preds.csv')